In [1]:
import pandas as pd
import json
import os


In [5]:


path = "D:\\phonepe_project\\pulse\\data\\aggregated\\transaction\\country\\india\\state"
path_a = os.listdir(path)

In [12]:
std = {'state':[], 'year':[],'quater':[],'transaction_type':[], 'transaction_count':[], 'transaction_amount':[]}

for i in path_a:
    path_yr = path+"\\"+i+"\\"
    year = os.listdir(path_yr)

    for j in year:
        path_file = path_yr + j
        path_n = os.listdir(path_file)

        for k in path_n:
            asd = path_file + "\\" + k
            data_n = open(asd, "r")
            output = json.load(data_n)

            for l in output['data']['transactionData']:
                name = l['name']
                count = l['paymentInstruments'][0]['count']
                amount = l['paymentInstruments'][0]['amount']
                std['transaction_type'].append(name)
                std['transaction_count'].append(count)
                std['transaction_amount'].append(amount)
                std['state'].append(i)
                std['year'].append(j)
                std['quater'].append(int(k.strip('.json')))

df = pd.DataFrame(std)


In [13]:
df

,state,year,quater,transaction_type,transaction_count,transaction_amount
0,andaman-&-nicobar-islands,2018,1,Recharge & bill payments,4200,1.845307e+06
1,andaman-&-nicobar-islands,2018,1,Peer-to-peer payments,1871,1.213866e+07
2,andaman-&-nicobar-islands,2018,1,Merchant payments,298,4.525072e+05
3,andaman-&-nicobar-islands,2018,1,Financial Services,33,1.060142e+04
4,andaman-&-nicobar-islands,2018,1,Others,256,1.846899e+05
...,...,...,...,...,...,...
4129,west-bengal,2023,3,Merchant payments,296965002,2.004404e+11
4130,west-bengal,2023,3,Peer-to-peer payments,271150280,8.310501e+11
4131,west-bengal,2023,3,Recharge & bill payments,63055684,3.920985e+10
4132,west-bengal,2023,3,Financial Services,389751,4.249856e+08


In [14]:
df.to_csv(r'D:\phonepe_project\agg_trans.csv')

In [16]:
import psycopg2

mydb = psycopg2.connect(
    host = "localhost",
    database = "postgres",
    user = "postgres",
    password = "123456")

mycursor = mydb.cursor()

In [23]:
query1 = """
    COPY agg_trans(state, year, quater, transaction_type, transaction_count, transaction_amount)
    FROM 'D:\\phonepe_project\\agg_trans.csv'
    DELIMITER ','
    CSV HEADER
"""


try:
    mycursor.execute(query1)
    mydb.commit()
except Exception as e:
    print("Error:", e)
    mydb.rollback()

Error: extra data after last expected column
CONTEXT:  COPY agg_trans, line 2: "0,andaman-&-nicobar-islands,2018,1,Recharge & bill payments,4200,1845307.4673655091"



In [4]:
import pandas as pd
from sqlalchemy import create_engine

# PostgreSQL database connection parameters
db_config = {
    'user': 'postgres',
    'password': '123456',
    'host': 'localhost',
    'port': '5432',
    'database': 'postgres',
}

# Create a SQLAlchemy engine
engine = create_engine(f'postgresql+psycopg2://{db_config["user"]}:{db_config["password"]}@{db_config["host"]}:{db_config["port"]}/{db_config["database"]}')

# CSV file path
csv_file_path = 'D:\\phonepe_project\\agg_trans.csv'

# Read CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# Insert data into PostgreSQL table
try:
    df.to_sql('agg_trans', con=engine, if_exists='replace', index=False)
    print("Data inserted successfully.")
except Exception as e:
    print("Error:", e)
finally:
    engine.dispose()  # Close the database connection


Data inserted successfully.
